In [84]:
import datetime
import os
import re
import subprocess
import time

In [76]:
def listMacAddresses():
    process = subprocess.Popen('airport -s', shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process.wait()
    processOutput = process.stdout.read().decode('utf-8')

    lines = list(filter(lambda x: len(x) > 0, processOutput.split('\n')))
    header = lines[0]
    reHeader = re.compile('\s+SSID')
    networkNameLength = len(reHeader.search(header).group())
    parsedValues = [parseAirportLine(line, networkNameLength) for line in lines]
    
    return parsedValues

In [77]:
def parseAirportLine(line, networkNameLength):
    if len(line) == 0:
        return

    name = re.sub('\s+', ' ', line[:networkNameLength]).strip()
    regex = re.compile('\s+')
    bssid = regex.split(line[networkNameLength:])[1]

    return {
        'name': name,
        'bssid': bssid,
    }

In [80]:
addresses = listMacAddresses()
list(map(lambda x: ';'.join(x.values()), addresses))

['SSID;BSSID',
 "Livebox-rollin's;2c:39:96:49:d3:c6",
 'Bbox-7887CADF;e8:be:81:aa:9f:10',
 'FreeWifi_secure;e4:9e:12:ef:a5:a6',
 'FreeWifi_secure;f4:ca:e5:dd:79:1a',
 'NUMERICABLE-A288;c0:d9:62:31:f7:89',
 'FreeWifi;00:24:d4:95:61:d0',
 'Freebox-9561CE;00:24:d4:95:61:cf',
 'freebox_HUGUTI_2GEXT;b0:b9:8a:4d:bd:c5',
 'Bbox-109AD9;00:1f:9f:bd:e3:5d',
 'FreeWifi;f4:ca:e5:dd:79:19',
 'Chezjul;f4:ca:e5:dd:79:18',
 'SFR-2017;44:e9:dd:ed:d8:3e',
 'Bbox-10C10234;3c:81:d8:75:65:90',
 'FreeWifi_secure;f4:ca:e5:d2:f2:0e',
 'Livebox-8EEA;a0:1b:29:a7:8e:ea',
 'orange;12:7c:b2:14:54:e3',
 'FreeWifi;f4:ca:e5:d2:f2:0d',
 'Livebox-54E3;30:7c:b2:14:54:e3',
 'FreeWifi_secure;68:a3:78:ad:9e:52',
 'Coloc-Soupe;6c:38:a1:5a:dd:78',
 'Bbox-C974D670;f8:ab:05:06:45:a0',
 'SFR-5b10;a4:08:f5:c9:5b:16',
 'Bbox-F92A10;00:1f:9f:bf:09:69',
 'Bbox-41AA8A4E-5GHz;64:7c:34:d0:e4:d0',
 'SFR-5b10_5GHz;a4:08:f5:c9:5b:17',
 'Bbox-C974D670-5GHz;00:26:86:1e:38:5e']

In [95]:
fileName = './logs.txt'
readingMode = 'a'
if not os.path.exists(fileName):
    readingMode = 'w+'

while True:
    with open(fileName, readingMode) as logFile:
        addresses = listMacAddresses()
        currentTime = str(datetime.datetime.now())
        logs = list(map(lambda x: ';'.join([currentTime] + list(x.values()) + ['\n']), addresses[1:]))
        print('{} - Writting logs...'.format(currentTime))
        logFile.write(''.join(logs))
        logFile.close()

    time.sleep(15)

2018-02-04 16:52:08.603282 - Writting logs...
2018-02-04 16:52:27.592375 - Writting logs...


KeyboardInterrupt: 